<a href="https://colab.research.google.com/github/dAn-solution/competition/blob/main/Prob_kiva_015.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Kiva／クラウドファンディングの資金調達額予測
- ランダムフォレスト を実施

### Google Driveのマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/'My Drive'

Mounted at /content/drive
/content/drive/My Drive


### ライブラリのインストール、インポート

In [ ]:
# カレントディレクトリを変更
import os
os.chdir('/content/drive/My Drive/Probdata/kiva/')
print(os.getcwd())

/content/drive/My Drive/Probdata/kiva


In [ ]:
class Config():
    root_path = './'
    input_path = os.path.join(root_path, 'input')
    output_path = os.path.join(root_path, 'output')
    result_path = os.path.join(root_path, 'result')
    bert_model_name = 'bert-base-uncased'
    seed = 42
    debug = False

In [ ]:
import pandas as pd
import numpy as np
import datetime
import warnings

import matplotlib.pyplot as plt

import sklearn
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor as RFR

### データの読み込み

In [ ]:
train_df = pd.read_csv(f'{Config.input_path}/train.csv')
test_df = pd.read_csv(f'{Config.input_path}/test.csv')
sample_df = pd.read_csv(f'{Config.input_path}/sample_submission.csv')

train_tags_pca = np.load(f'{Config.result_path}/train_tags_pca.npy')
test_tags_pca = np.load(f'{Config.result_path}/test_tags_pca.npy')
train_bert_pca = np.load(f'{Config.result_path}/train_bert_pca.npy')
test_bert_pca = np.load(f'{Config.result_path}/test_bert_pca.npy')
train_bert_018_pca = np.load(f'{Config.result_path}/train_bert_018_pca.npy')
test_bert_018_pca = np.load(f'{Config.result_path}/test_bert_018_pca.npy')

train_X_pca = np.load(f'{Config.result_path}/train_X_pca.npy')
test_X_pca = np.load(f'{Config.result_path}/test_X_pca.npy')

## ランダムフォレストの実行

In [ ]:
# # trainデータを目的変数と説明変数に分割
# train_columns = ["ORIGINAL_LANGUAGE", "ACTIVITY_NAME", "SECTOR_NAME", "COUNTRY_CODE","CURRENCY_POLICY",
#        "CURRENCY","REPAYMENT_INTERVAL", "DISTRIBUTION_MODEL"]
target_column = "LOAN_AMOUNT"

y = train_df[target_column]
# X = train_df[train_columns]
# test_X = test_df[train_columns]

In [ ]:
train_pca = np.concatenate([train_X_pca, train_tags_pca, train_bert_pca, train_bert_018_pca], axis=1)
test_pca = np.concatenate([test_X_pca, test_tags_pca, test_bert_pca, test_bert_018_pca], axis=1)

# X = pd.concat([X, pd.DataFrame(train_pca)], axis=1)
# test_X = pd.concat([test_X, pd.DataFrame(test_pca)], axis=1)
X = pd.DataFrame(train_pca)
test_X = pd.DataFrame(test_pca)
print(X.shape)
print(test_X.shape)

(91333, 18)
(91822, 18)


In [ ]:
# trainデータを訓練用とテスト用に分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = Config.seed)

In [ ]:
# LGBM用のパラメータ設定
# lgb_params = {
#     "application" : "regression_l1",
#     'n_estimators': 500, 'learning_rate': 0.1,
#     "metric" : "l1" #mae
# }

# lgb_params = {
#     "application" : "regression_l1",
#     # 'objective': 'regression',
#     'importance_type': 'gain',
#     'metric': 'l1', # mae
#     'seed': Config.seed,
#     'n_jobs': -1,
#     'verbose': 0,

#     'n_estimators': 10000, 'learning_rate': 0.09,
#     'boosting_type': 'gbdt',
#     'subsample': 0.5, 'subsample_freq': 1,
#     'colsample_bytree': 0.5,
#     'num_leaves': 256, 'max_depth': -1, 'min_child_samples': 255,
#     'max_bin': 256,
# }

In [ ]:
# 学習/予測

y_oof = np.zeros(len(y_train))
meta_preds = []
y_preds = []
kf = StratifiedKFold(n_splits=5)
for fold, (tr_idx, vl_idx) in enumerate(kf.split(X_train, y_train)):
    x_tr_fold = X.iloc[tr_idx]
    y_tr_fold = y.iloc[tr_idx]
    x_vl_fold = X.iloc[vl_idx]
    y_vl_fold = y.iloc[vl_idx]

    model = RFR(n_estimators=10)
    model.fit(x_tr_fold, y_tr_fold)
    # model.fit(
    #     x_tr_fold, y_tr_fold,
    #     eval_set=(x_vl_fold, y_vl_fold),
    #     eval_metric='l1',
    #     verbose=False,
    #     early_stopping_rounds=100,
    # )
    y_oof[vl_idx] = model.predict(x_vl_fold)
    meta_preds.append(model.predict(X))
    y_preds.append(model.predict(test_X))
    print(
        f'fold {fold} score:', mean_absolute_error(y_oof[vl_idx], y_vl_fold)
    )

print('X score:', mean_absolute_error(model.predict(X), y))

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,


fold 0 score: 387.69963733406325
fold 1 score: 370.425306234175
fold 2 score: 367.3795592965168
fold 3 score: 371.4061109970574
fold 4 score: 379.9047081365907
X score: 231.78341891758728


In [ ]:
# メタモデルへの引き渡し
train_015_df = pd.DataFrame(np.mean(meta_preds, axis=0))
train_015_df.to_csv(os.path.join(Config.result_path, "train_015.csv"), index=False)

test_015_df = pd.DataFrame(np.mean(y_preds, axis=0))
test_015_df.to_csv(os.path.join(Config.result_path, "test_015.csv"), index=False)

In [ ]:
test_015_df.describe()

,0
count,91822.000000
mean,673.345217
std,566.570719
min,65.500000
25%,368.000000
50%,507.500000
75%,760.000000
max,7516.000000


### 提出データの作成

In [ ]:
sample_df["LOAN_AMOUNT"] = np.mean(y_preds, axis=0)
print(sample_df)

       LOAN_ID  LOAN_AMOUNT
0      2041445        422.5
1      1944435        813.0
2      2083354        399.0
3      1993565        723.5
4      2064272        244.0
...        ...          ...
91817  1993862        357.0
91818  2015070        261.5
91819  1950349        645.5
91820  1921580       1531.5
91821  1976733        625.0

[91822 rows x 2 columns]


In [ ]:
# submitファイルのファイル名に利用する作成時刻の取得

now = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=1)))
date_time = "{0:%Y%m%d_%H%M}".format(now)

In [ ]:
# submitファイルの出力(Google Driveに出力）
# 形式：submit_yyyymmdd_hhmm.csv

sample_df.to_csv(os.path.join(Config.output_path, f"submit_{date_time}.csv"), index=False)
